In [2]:
from keras.applications import VGG16
conv_base = VGG16(weights='imagenet',include_top=False,input_shape=(90, 80, 3))

In [3]:
from keras.models import load_model
model = load_model('V1_FrozenVGG_Small.h5')
#model.load_weights('V1_FrozenVGG_Small_Weights.h5')

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               524544    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 15,239,489
Trainable params: 7,604,225
Non-trainable params: 7,635,264
_________________________________________________________________


In [5]:
img_path = 'image.png'
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

In [6]:
img = image.load_img(img_path, target_size=(90,80))

img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

print(img_tensor.shape)

(1, 90, 80, 3)


In [7]:
import matplotlib.pyplot as plt
plt.imshow(img_tensor[0])
plt.show()

In [8]:
preds = model.predict(x)
preds

array([[ 0.]], dtype=float32)

In [9]:
argmax = np.argmax(preds[0])
argmax

0

In [10]:
output = model.output[:,argmax]
print(output.shape)
print(output[0])


(?,)
Tensor("strided_slice_1:0", shape=(), dtype=float32)


In [11]:
vgg16_Model = model.get_layer('vgg16')
last_conv_layer = vgg16_Model.get_layer('block5_conv3')
print(last_conv_layer)


In [12]:
print(last_conv_layer.output[0])

Tensor("strided_slice_2:0", shape=(5, 5, 512), dtype=float32)


In [13]:
from keras import backend as K
grads = K.gradients(output, last_conv_layer.output)[0]
print(grads)

None


In [14]:
pooled_grads = K.mean(grads, axis=(0, 1, 2))
pooled_grads

AttributeError: 'NoneType' object has no attribute 'dtype'

In [ ]:
iterate = K.function([model.input],[pooled_grads, last_conv_layer.output[0]])

In [ ]:
pooled_grads_value, conv_layer_output_value = iterate([x])

for i in range(128):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

heatmap = np.mean(conv_layer_output_value, axis=-1)

In [ ]:
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)

In [ ]:
import matplotlib.pyplot as plt
plt.matshow(heatmap)

In [ ]:
import cv2
img = cv2.imread(img_path)
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
superimposed_img = heatmap * 0.4 + img

In [ ]:
CAM_output = cv2.imwrite('Image_CAM.png', superimposed_img)
out_img_path = 'image.png'
out_img = image.load_img(out_img_path, target_size=(90, 80))
out_img_tensor = image.img_to_array(out_img)
out_img_tensor = np.expand_dims(out_img_tensor, axis=0)
out_img_tensor /= 255.

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img_tensor[0])
plt.show()